In [1]:
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-19"
os.environ["SPARK_HOME"] = "C:/spark/spark-3.3.2-bin-hadoop3"

In [2]:
import pyspark 
from pyspark.sql import SparkSession
jdbc_driver_path= "C:/Users/Sharing Vision/anaconda3/libs/postgresql-42.7.3.jar"
spark = SparkSession.builder.appName("PySpark")\
        .config("spark.jars.packages", "org.postgresql:postgresql:42.7.3")\
        .config('spark.dynamicAllocation.enabled', 'false')\
        .config('spark.executor.instances', '8')\
        .config('spark.executor.cores', '6')\
        .config('spark.executor.memory', '12g')\
        .getOrCreate()
spark

  

In [3]:

#ingest table categories

ingest=spark.read.format("jdbc")\
      .option("driver","org.postgresql.Driver")
 
dst_db_ip="localhost"
dst_db_port="5432"
dst_db_name="northwind"
user="postgres"
password="12345678"
dst_schm_name="public"
dst_tbl_name="categories"

jdbc_url = f"jdbc:postgresql://{dst_db_ip}:{dst_db_port}/{dst_db_name}?user={user}&password={password}" 

df_categories = spark.read.format("jdbc") \
      .option("driver", "org.postgresql.Driver") \
      .option("url", jdbc_url) \
      .option("dbtable", f"(select * from {dst_schm_name}.{dst_tbl_name}) as tmp") \
      .load()

df_categories.createOrReplaceTempView("df_categories")
df_categories.show()


# df=ingest\
#   .option("url","jdbc:postgresql://{}:{}/{}?user={}&password={}".format(dst_db_ip,dst_db_port,dst_db_name,user,password))\
#   .option("dbtable","(select * from {}.{} ) as tmp".format(dst_schm_name,dst_tbl_name))\
#   .load()


+----------+--------------+--------------------+--------------------+
|CATEGORYID|  CATEGORYNAME|         DESCRIPTION|             PICTURE|
+----------+--------------+--------------------+--------------------+
|         1|     Beverages|Soft drinks coffe...|0x151C2F000200000...|
|         2|    Condiments|Sweet and savory ...|0x151C2F000200000...|
|         3|   Confections|Desserts candies ...|0x151C2F000200000...|
|         4|Dairy Products|             Cheeses|0x151C2F000200000...|
|         5|Grains/Cereals|Breads crackers p...|0x151C2F000200000...|
|         6|  Meat/Poultry|      Prepared meats|0x151C2F000200000...|
|         7|       Produce|Dried fruit and b...|0x151C2F000200000...|
|         8|       Seafood|    Seaweed and fish|0x151C2F000200000...|
+----------+--------------+--------------------+--------------------+



In [4]:
#Ingest order_details

dst_tbl_name="order_details"
df_order_details = spark.read.format("jdbc") \
      .option("driver", "org.postgresql.Driver") \
      .option("url", jdbc_url) \
      .option("dbtable", f"(select * from {dst_schm_name}.{dst_tbl_name}) as tmp") \
      .load()
df_order_details.createOrReplaceTempView("df_order_details")     
df_order_details.show()

+-------+---------+---------+--------+--------+
|ORDERID|PRODUCTID|UNITPRICE|QUANTITY|DISCOUNT|
+-------+---------+---------+--------+--------+
|  10248|       11|     14.0|      12|     0.0|
|  10248|       42|      9.8|      10|     0.0|
|  10248|       72|     34.8|       5|     0.0|
|  10249|       14|     18.6|       9|     0.0|
|  10249|       51|     42.4|      40|     0.0|
|  10250|       41|      7.7|      10|     0.0|
|  10250|       51|     42.4|      35|    0.15|
|  10250|       65|     16.8|      15|    0.15|
|  10251|       22|     16.8|       6|    0.05|
|  10251|       57|     15.6|      15|    0.05|
|  10251|       65|     16.8|      20|     0.0|
|  10252|       20|     64.8|      40|    0.05|
|  10252|       33|      2.0|      25|    0.05|
|  10252|       60|     27.2|      40|     0.0|
|  10253|       31|     10.0|      20|     0.0|
|  10253|       39|     14.4|      42|     0.0|
|  10253|       49|     16.0|      40|     0.0|
|  10254|       24|      3.6|      15|  

In [5]:
#Ingest products

dst_tbl_name="products"
df_products = spark.read.format("jdbc") \
      .option("driver", "org.postgresql.Driver") \
      .option("url", jdbc_url) \
      .option("dbtable", f"(select * from {dst_schm_name}.{dst_tbl_name}) as tmp") \
      .load()
df_products.createOrReplaceTempView("df_products")      
df_products.show()

+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|PRODUCTID|         PRODUCTNAME|SUPPLIERID|CATEGORYID|     QUANTITYPERUNIT|UNITPRICE|UNITSINSTOCK|UNITSONORDER|REORDERLEVEL|DISCONTINUED|
+---------+--------------------+----------+----------+--------------------+---------+------------+------------+------------+------------+
|        1|                Chai|         1|         1|  10 boxes x 20 bags|     18.0|          39|           0|          10|           0|
|        2|               Chang|         1|         1|  24 - 12 oz bottles|     19.0|          17|          40|          25|           0|
|        3|       Aniseed Syrup|         1|         2| 12 - 550 ml bottles|     10.0|          13|          70|          25|           0|
|        4|Chef Anton's Caju...|         2|         2|      48 - 6 oz jars|     22.0|          53|           0|           0|           0|
|        5|Chef Anton's Gumb...|  

In [6]:
#Ingest orders

dst_tbl_name="orders"
df_orders = spark.read.format("jdbc") \
      .option("driver", "org.postgresql.Driver") \
      .option("url", jdbc_url) \
      .option("dbtable", f"(select * from {dst_schm_name}.{dst_tbl_name}) as tmp") \
      .load()
df_orders.createOrReplaceTempView("df_orders")
df_orders.show()

+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+--------------------+--------------+----------+--------------+-----------+
|ORDERID|CUSTOMERID|EMPLOYEEID| ORDERDATE|REQUIREDDATE|SHIPPEDDATE|SHIPVIA|FREIGHT|            SHIPNAME|         SHIPADDRESS|      SHIPCITY|SHIPREGION|SHIPPOSTALCODE|SHIPCOUNTRY|
+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+--------------------+--------------+----------+--------------+-----------+
|  10248|     VINET|         5|1996-07-04|  1996-08-01| 1996-07-16|      3|  32.38|Vins et alcools C...|  59 rue de l'Abbaye|         Reims|      null|         51100|     France|
|  10249|     TOMSP|         6|1996-07-05|  1996-08-16| 1996-07-10|      1|  11.61|  Toms Spezialitäten|       Luisenstr. 48|       Münster|      null|         44087|    Germany|
|  10250|     HANAR|         4|1996-07-08|  1996-08-05| 1996-07-12|      2|  65.83|       Hanari Carnes| 

In [7]:
#Ingest suppliers

dst_tbl_name="suppliers"
df_suppliers = spark.read.format("jdbc") \
      .option("driver", "org.postgresql.Driver") \
      .option("url", jdbc_url) \
      .option("dbtable", f"(select * from {dst_schm_name}.{dst_tbl_name}) as tmp") \
      .load()
df_suppliers.createOrReplaceTempView("df_suppliers")
df_suppliers.show()

+----------+--------------------+--------------------+--------------------+--------------------+-----------+--------+----------+---------+---------------+---------------+--------------------+
|SUPPLIERID|         COMPANYNAME|         CONTACTNAME|        CONTACTTITLE|             ADDRESS|       CITY|  REGION|POSTALCODE|  COUNTRY|          PHONE|            FAX|            HOMEPAGE|
+----------+--------------------+--------------------+--------------------+--------------------+-----------+--------+----------+---------+---------------+---------------+--------------------+
|         1|      Exotic Liquids|    Charlotte Cooper|  Purchasing Manager|      49 Gilbert St.|     London|    null|   EC1 4SD|       UK| (171) 555-2222|           null|                null|
|         2|New Orleans Cajun...|       Shelley Burke| Order Administrator|      P.O. Box 78934|New Orleans|      LA|     70117|      USA| (100) 555-4822|           null|         #CAJUN.HTM#|
|         3|Grandma Kelly's H...|       

In [8]:
#Ingest employees

dst_tbl_name="employees"
df_employees = spark.read.format("jdbc") \
      .option("driver", "org.postgresql.Driver") \
      .option("url", jdbc_url) \
      .option("dbtable", f"(select * from {dst_schm_name}.{dst_tbl_name}) as tmp") \
      .load()
df_employees.createOrReplaceTempView("df_employees")
df_employees.show()

+----------+---------+---------+--------------------+---------------+----------+----------+--------------------+--------+------+----------+-------+--------------+---------+--------------------+--------------------+---------+--------------------+
|EMPLOYEEID| LASTNAME|FIRSTNAME|               TITLE|TITLEOFCOURTESY| BIRTHDATE|  HIREDATE|             ADDRESS|    CITY|REGION|POSTALCODE|COUNTRY|     HOMEPHONE|EXTENSION|               PHOTO|               NOTES|REPORTSTO|           PHOTOPATH|
+----------+---------+---------+--------------------+---------------+----------+----------+--------------------+--------+------+----------+-------+--------------+---------+--------------------+--------------------+---------+--------------------+
|         1|  Davolio|    Nancy|Sales Representative|            Ms.|1948-12-08|1992-05-01|507 20th Ave. E. ...| Seattle|    WA|     98122|    USA|(206) 555-9857|     5467|0x151C2F000200000...|Education include...|      2.0|http://accweb/emm...|
|         2|   F

In [9]:
# Create datamart tables  (employee_revenues)


employee_revenues = spark.sql("""

WITH employee_revenue AS (
  SELECT
    e.FIRSTNAME || ' ' || e.LASTNAME AS employee_name,
    date_format(DATE_TRUNC('month', o.ORDERDATE), 'yyy-MM')  AS month_order,
    SUM((od.UNITPRICE - (od.UNITPRICE * od.DISCOUNT)) * od.QUANTITY) AS gross_revenue
  FROM
    df_order_details od
    JOIN df_orders o ON od.ORDERID = o.ORDERID
    JOIN df_employees e ON o.EMPLOYEEID = e.EMPLOYEEID
  GROUP BY
    employee_name, month_order
),
ranked_employees AS (
  SELECT
    employee_name,
    month_order,
    gross_revenue,
    ROW_NUMBER() OVER (PARTITION BY month_order ORDER BY gross_revenue DESC) AS rank
  FROM
    employee_revenue
)
SELECT
  employee_name,
  month_order,
  gross_revenue
FROM
  ranked_employees
WHERE
  rank = 1

""")

In [10]:
#cek bentuk data

employee_revenues.show()

+----------------+-----------+------------------+
|   employee_name|month_order|     gross_revenue|
+----------------+-----------+------------------+
|Margaret Peacock|    1996-07|          11860.45|
|  Laura Callahan|    1996-08|            8263.8|
|   Nancy Davolio|    1996-09|            6651.0|
|Margaret Peacock|    1996-10|          13718.97|
|Margaret Peacock|    1996-11|11311.419999999998|
| Steven Buchanan|    1996-12|10030.820000000003|
|Margaret Peacock|    1997-01|23736.465000000004|
|Margaret Peacock|    1997-02|         12121.995|
| Janet Leverling|    1997-03|11599.400000000001|
|Margaret Peacock|    1997-04|          13475.99|
| Janet Leverling|    1997-05|           18049.6|
|   Andrew Fuller|    1997-06| 6882.199999999999|
|   Nancy Davolio|    1997-07|          19530.93|
|Margaret Peacock|    1997-08|          16485.54|
|     Robert King|    1997-09|          13249.86|
|   Nancy Davolio|    1997-10|          12414.15|
| Janet Leverling|    1997-11|          9598.084|


In [11]:
#ubah ke bentuk pandas untuk di upload ke BQ
df_datamart_employee_revenues=employee_revenues.toPandas()

C:\Users\Sharing Vision\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Sharing Vision\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [12]:
# Create datamart tables  (category_sales)


category_sales = spark.sql("""

WITH category_sales AS (
  SELECT
    c.CATEGORYNAME AS category_name,
    DATE_FORMAT(DATE_TRUNC('month', o.ORDERDATE), 'yyyy-MM') AS month_order,
    SUM(od.QUANTITY) AS total_sold
  FROM
    df_order_details od
    JOIN df_orders o ON od.ORDERID = o.ORDERID
    JOIN df_products p ON od.PRODUCTID = p.PRODUCTID
    JOIN df_categories c ON p.CATEGORYID = c.CATEGORYID
  GROUP BY
    category_name, month_order
),
ranked_categories AS (
  SELECT
    category_name,
    month_order,
    total_sold,
    ROW_NUMBER() OVER (PARTITION BY month_order ORDER BY total_sold DESC) AS rank
  FROM
    category_sales
)

SELECT
  category_name,
  month_order,
  total_sold
FROM
  ranked_categories
WHERE
  rank = 1

""")

In [13]:
#cek bentuk data

category_sales.show()

+--------------+-----------+----------+
| category_name|month_order|total_sold|
+--------------+-----------+----------+
|Dairy Products|    1996-07|       304|
|     Beverages|    1996-08|       347|
|     Beverages|    1996-09|       285|
|Dairy Products|    1996-10|       367|
|Dairy Products|    1996-11|       372|
|Dairy Products|    1996-12|       653|
|   Confections|    1997-01|       623|
|   Confections|    1997-02|       425|
|     Beverages|    1997-03|       471|
|   Confections|    1997-04|       394|
|Dairy Products|    1997-05|       494|
|Dairy Products|    1997-06|       405|
|Dairy Products|    1997-07|       374|
|   Confections|    1997-08|       335|
|       Seafood|    1997-09|       630|
|     Beverages|    1997-10|       491|
|       Seafood|    1997-11|       414|
|     Beverages|    1997-12|       498|
|       Seafood|    1998-01|       735|
|     Beverages|    1998-02|       834|
+--------------+-----------+----------+
only showing top 20 rows



In [14]:
#ubah ke bentuk pandas untuk di upload ke BQ
df_datamart_category_sales=category_sales.toPandas()

In [15]:
# Create datamart tables  (supplier_revenue)


supplier_revenue = spark.sql("""

WITH supplier_revenue AS (
  SELECT
    s.COMPANYNAME AS company_name,
    DATE_FORMAT(DATE_TRUNC('month', o.ORDERDATE), 'yyyy-MM') AS month_order,
    SUM((od.UNITPRICE - (od.UNITPRICE * od.DISCOUNT)) * od.QUANTITY) AS gross_revenue
  FROM
    df_order_details od
    JOIN df_orders o ON od.ORDERID = o.ORDERID
    JOIN df_products p ON od.PRODUCTID = p.PRODUCTID
    JOIN df_suppliers s ON p.SUPPLIERID = s.SUPPLIERID
  GROUP BY
    company_name, month_order
)

SELECT
  company_name,
  month_order,
  gross_revenue
FROM
  supplier_revenue
ORDER BY
  company_name, month_order

""")

In [16]:
#cek bentuk data

supplier_revenue.show()

+--------------------+-----------+-----------------+
|        company_name|month_order|    gross_revenue|
+--------------------+-----------+-----------------+
|Aux joyeux ecclés...|    1996-07|            691.2|
|Aux joyeux ecclés...|    1996-09|           1252.8|
|Aux joyeux ecclés...|    1996-10|           4062.8|
|Aux joyeux ecclés...|    1996-11|         15857.04|
|Aux joyeux ecclés...|    1996-12|6568.800000000001|
|Aux joyeux ecclés...|    1997-01|         18803.36|
|Aux joyeux ecclés...|    1997-02|            374.4|
|Aux joyeux ecclés...|    1997-03|           6540.0|
|Aux joyeux ecclés...|    1997-04|           4132.5|
|Aux joyeux ecclés...|    1997-05|           8853.6|
|Aux joyeux ecclés...|    1997-06|            180.0|
|Aux joyeux ecclés...|    1997-07|         3844.875|
|Aux joyeux ecclés...|    1997-08|            643.5|
|Aux joyeux ecclés...|    1997-09|          3924.45|
|Aux joyeux ecclés...|    1997-12|           6377.1|
|Aux joyeux ecclés...|    1998-01|         190

In [17]:
#ubah ke bentuk pandas untuk di upload ke BQ
df_datamart_supplier_revenue=supplier_revenue.toPandas()

In [18]:

from google.cloud import bigquery
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./bridgtl-bgd-d-prj-bigdata-e3ffa0f77953.json"
client = bigquery.Client()

In [ ]:
# # Upload supplier_revenue to Bigquery
# # TABEL TUJUAN DI GCP
# table_id = "isi_nama_project.isi_nama_schema.supplier_revenue"

# client = bigquery.Client()

# job_config = bigquery.LoadJobConfig(
# source_format=bigquery.SourceFormat.CSV, autodetect=False,write_disposition = "WRITE_TRUNCATE") # bisa truncate bisa append

# job = client.load_table_from_dataframe(
# df_datamart_supplier_revenue, table_id, job_config=job_config)
# job.result()

In [ ]:
# Upload category_sales to Bigquery
# TABEL TUJUAN DI GCP
table_id = "isi_nama_project.isi_nama_schema.category_sales"

client = bigquery.Client()

job_config = bigquery.LoadJobConfig(
source_format=bigquery.SourceFormat.CSV, autodetect=False,write_disposition = "WRITE_TRUNCATE") # bisa truncate bisa append

job = client.load_table_from_dataframe(
df_datamart_category_sales, table_id, job_config=job_config)
job.result()

In [ ]:
# # Upload employee_revenues to Bigquery
# # TABEL TUJUAN DI GCP
# table_id = "isi_nama_project.isi_nama_schema.employee_revenues"

# client = bigquery.Client()

# job_config = bigquery.LoadJobConfig(
# source_format=bigquery.SourceFormat.CSV, autodetect=False,write_disposition = "WRITE_TRUNCATE") # bisa truncate bisa append

# job = client.load_table_from_dataframe(
# df_datamart_employee_revenues, table_id, job_config=job_config)
# job.result()